# 1. Importing Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\samir\OneDrive\Documents\10-2022 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v2.pkl'))

# 2. Task

### Q2.  Find the aggregated mean of the “order_number” column grouped by “department_id”

In [3]:
# Group By
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Q3. Analyze the result. How do the results for the entire dataframe differ from those of the subset?

#### Running this function on the full dataframe allowed all of the department_id's to be shown. The means shown in the full dataframe are distributed more evenly versus the subset means. 

### Q4. Create a loyalty flag for existing customers using the transform() and loc() functions.

In [6]:
# Group by user_id, and tranform order_number to generate max order numbers for each user
ords_prods_merge['max_order']= ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [13]:
# Checking for new column
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer
5,3367565,1,6,2,7,19.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
6,550135,1,7,1,9,20.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer
7,3108588,1,8,1,14,14.0,False,196,2,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer
8,2295261,1,9,1,16,0.0,False,196,4,1,...,77,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer
9,2550362,1,10,4,8,30.0,False,196,1,1,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer


In [8]:
# Defining conditions for loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Checking loyalty flag frequency
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [12]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(15)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


### Q5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer).

In [14]:
# Group by loyalty flag and run descriptive statistics
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### Q6. Create a spending flag for each user based on the average price across all their orders

In [15]:
# Creating average price column
ords_prods_merge['average_price']= ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
# Defining conditions for spending flag
ords_prods_merge.loc[ords_prods_merge['average_price']<10, 'spending_flag']= 'Low spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['average_price']>=10, 'spending_flag']= 'High spender'

In [18]:
# Spending flag frequency
ords_prods_merge['spending_flag'].value_counts(dropna= False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

In [19]:
ords_prods_merge[['user_id', 'loyalty_flag', 'average_price', 'spending_flag']].head(15)

,user_id,loyalty_flag,average_price,spending_flag
0,1,New customer,6.367797,Low spender
1,1,New customer,6.367797,Low spender
2,1,New customer,6.367797,Low spender
3,1,New customer,6.367797,Low spender
4,1,New customer,6.367797,Low spender
5,1,New customer,6.367797,Low spender
6,1,New customer,6.367797,Low spender
7,1,New customer,6.367797,Low spender
8,1,New customer,6.367797,Low spender
9,1,New customer,6.367797,Low spender


### Q7. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [20]:
# Creating order frequenct flag
ords_prods_merge['median_order_frequency']= ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
# Defining conditions for spending flag
ords_prods_merge.loc[ords_prods_merge['median_order_frequency']>20, 'order_frequency_flag']= 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_order_frequency']>10) & (ords_prods_merge['median_order_frequency']<=20), 'order_frequency_flag']= 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_order_frequency']<=10, 'order_frequency_flag']= 'Frequent customer'

In [22]:
#frequency of order frequency flag
ords_prods_merge['order_frequency_flag'].value_counts(dropna= False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency_flag, dtype: int64

In [23]:
ords_prods_merge[['user_id', 'loyalty_flag', 'average_price', 'spending_flag', 'order_frequency_flag']].head(15)

,user_id,loyalty_flag,average_price,spending_flag,order_frequency_flag
0,1,New customer,6.367797,Low spender,Non-frequent customer
1,1,New customer,6.367797,Low spender,Non-frequent customer
2,1,New customer,6.367797,Low spender,Non-frequent customer
3,1,New customer,6.367797,Low spender,Non-frequent customer
4,1,New customer,6.367797,Low spender,Non-frequent customer
5,1,New customer,6.367797,Low spender,Non-frequent customer
6,1,New customer,6.367797,Low spender,Non-frequent customer
7,1,New customer,6.367797,Low spender,Non-frequent customer
8,1,New customer,6.367797,Low spender,Non-frequent customer
9,1,New customer,6.367797,Low spender,Non-frequent customer


In [24]:
# Exporting Dataframe
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v3.pkl'))